In [ ]:
%matplotlib widget

import cv2 as cv
import numpy as np
import yaml
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import IntText, VBox, HBox, Checkbox, FloatText

In [ ]:
objectPoints = []
imagePoints = []
IMAGE_SIZE = []
cameraMatrix = None
distCoeffs = None
flags = 0
criteria = None

TAB = {}

In [ ]:
uploader = widgets.FileUpload(accept='.yml', multiple=False)
display(uploader)

In [ ]:
calib, = yaml.safe_load(uploader.value[0].content.tobytes())
camera = calib['camera']
# focus = camera['focus']
# sx = camera['sx']
# sy = camera['sy']
# cx = camera['cx']
# cy = camera['cy']
# k1 = camera['k1']
# k2 = camera['k2']
# k3 = camera['k3']
# p1 = camera['p1']
# p2 = camera['p2']
image_width = camera['image_width'] if 'image_width' in camera else 0
image_height = camera['image_height'] if 'image_height' in camera else 0
IMAGE_SIZE = [image_width, image_height]

object_points = [np.array(p, np.float32) for p in calib['calib_obj']['object_points']]
image_points = [np.array(p, np.float32) for p in calib['calib_obj']['observ_points']]
# reproject_points = [np.array(p) for p in calib['calib_obj']['reproject_points']]
# rvecs = np.array(calib['calib_obj']['rvecs'])
# tvecs = np.array(calib['calib_obj']['tvecs'])
# diff_points = [observ_point - reproject_point for observ_point, reproject_point in zip(observ_points, reproject_points)]

npos = len(object_points) if len(image_points) == len(object_points) else None
# ylim_min, ylim_max = np.min(diff_points), np.max(diff_points)
# xlim_max = np.max([p.shape[0] for p in object_points])

In [ ]:
class ToggleMultipleButtons():
    def __init__(self, *, options, description=''):
        tbs = [widgets.ToggleButton(description=str(i), icon='check', value=True) for i in options]
        self.value = [t.value for t in tbs]
        for i, t in enumerate(tbs):
            f = self.handle_toggle_change(i)
            t.observe(f, names='value')
        box_layout = widgets.Layout(flex_flow='row wrap')
        box = widgets.Box(children=tbs, layout=box_layout)
        button_placehold = widgets.Button(
            description=description,
            disabled=True,
            tooltip=description
        )
        button_all = widgets.Button(
            description='all',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='',
            icon='plus-square'
        )
        button_all.on_click(self.select_all)
        button_none = widgets.Button(
            description='none',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='',
            icon='minus-square'
        )
        button_none.on_click(self.select_none)
        hbox = widgets.HBox([button_placehold, button_all, button_none])
        self.vbox = widgets.VBox([hbox, box])
        
        self.tbs = tbs
    
    def handle_toggle_change(self, index):
        def fn(change):
            self.value[index] = change.new
            if change.owner.value:
                change.owner.icon='check'
            else:
                change.owner.icon='close'
        return fn
        
    def _ipython_display_(self):
        display(self.vbox)
    
    def select_all(self, b):
        for t in self.tbs:
            t.value = True
    
    def select_none(self, b):
        for t in self.tbs:
            t.value = False

In [ ]:
t = ToggleMultipleButtons(options=[str(i) for i in range(npos)], description='Select:')

In [ ]:
w = IntText(description='Width', value=image_width, tooltip='Image width (columns)')
def fn(change):
    IMAGE_SIZE[0] = change['new']
w.observe(fn, names='value')

h = IntText(description='Height', value=image_height, tooltip='Image height (rows)')
def fn(change):
    IMAGE_SIZE[1] = change['new']
h.observe(fn, names='value')

VBox([w, h])


In [ ]:
print(IMAGE_SIZE)

In [ ]:
m0 = Checkbox(
    value=False,
    description='CALIB_USE_INTRINSIC_GUESS',
    layout={'width': 'max-content'}
)

m1 = Checkbox(
    value=False,
    description='CALIB_RATIONAL_MODEL'
)

m2 = Checkbox(
    value=False,
    description='CALIB_THIN_PRISM_MODEL'
)

m3 = Checkbox(
    value=False,
    description='CALIB_TILTED_MODEL'
)

TAB['General'] = VBox([m0, m1, m2, m3])

In [ ]:
fx = FloatText(description='$f_x$', value=1., tooltip='Focal length in x')
fy = FloatText(description='$f_y$', value=1., tooltip='Focal length in y')
cx = FloatText(description='$c_x$', tooltip='Center of coordinates in x')
cy = FloatText(description='$c_y$', tooltip='Center of coordinates in y')
c1 = Checkbox(
    value=False,
    description='CALIB_FIX_ASPECT_RATIO'
)

c2 = Checkbox(
    value=False,
    description='CALIB_FIX_FOCAL_LENGTH'
)

c3 = Checkbox(
    value=False,
    description='CALIB_FIX_PRINCIPAL_POINT'
)

def on_value_change(change):
    if change.new:
        fx.disabled = True
    else:
        fx.disabled = False
    
c1.observe(on_value_change, names='value')

def on_value_change(change):
    if change.new:
        fx.disabled = True
        fy.disabled = True
    else:
        if c1.value:
            pass
        else:
            fx.disabled = False
        fy.disabled = False

c2.observe(on_value_change, names='value')

def on_value_change(change):
    if change.new:
        cx.disabled = True
        cy.disabled = True
    else:
        cx.disabled = False
        cy.disabled = False

c3.observe(on_value_change, names='value')

TAB['Intrinsic matrix'] = HBox([VBox([fx, fy, cx, cy]), VBox([c1, c2, c3])])

In [ ]:
items = [widgets.Label(str(i)) for i in range(24)]
widgets.Box(items)

In [ ]:
des_k = ['$k_' + str(i) + '$' for i in range(1, 7)]
des_c = ['CALIB_FIX_K' + str(i) for i in range(1, 7)]
tip_k = ['Raidal coefficients: ' + str(i) for i in range(1, 7)]
disabled = [False] * 3 + [True] * 3 
k = [widgets.FloatText(description=des, disabled=d, tooltip=t) for des, d, t in zip(des_k, disabled, tip_k)]
c = [widgets.Checkbox(description=des, disabled=d) for des, d in zip(des_c, disabled)]

for kn, cn in zip(k, c):
    def on_value_change(change, ek=kn):
        if change.new:
            ek.disabled = True
        else:
            ek.disabled = False
    cn.observe(on_value_change, names='value')

TAB['Radial'] = widgets.VBox([widgets.HBox([wk, wc]) for wk, wc in zip(k, c)])

In [ ]:
p1 = widgets.FloatText(description='$p_1$', tooltip='Tangential coefficients: 1')
p2 = widgets.FloatText(description='$p_2$', tooltip='Tangential coefficients: 2')
c = widgets.Checkbox(description='CALIB_ZERO_TANGENT_DIST')
def fn(change):
    if change.new:
        p1.value = 0.
        p2.value = 0.
        p1.disabled = True
        p2.disabled = True
    else:
        p1.disabled = False
        p2.disabled = False

c.observe(fn, names='value')

TAB['Tangential'] = widgets.VBox([c, p1, p2])

In [ ]:
des_s = ['$s_' + str(i) + '$' for i in range(1, 5)]
tip_s = ['Thin prism coefficients: ' + str(i) for i in range(1, 5)]
s = [widgets.FloatText(description=des, tooltip=t) for des, t in zip(des_s, tip_s)]
c = widgets.Checkbox(description='CALIB_FIX_S1_S2_S3_S4')

def fn(change):
    if change.new:
        for w in s:
            w.disabled = True
    else:
        for w in s:
            w.disabled = False

c.observe(fn, names='value')
TAB['Thin prism'] = widgets.VBox([c] + s)

In [ ]:
tx = widgets.FloatText(description='$\\tau_x$', tooltip='Tilted coefficients: x')
ty = widgets.FloatText(description='$\\tau_y$', tooltip='Tilted coefficients: y')
c = widgets.Checkbox(description='CALIB_FIX_TAUX_TAUY')
def fn(change):
    if change.new:
        tx.disabled = True
        ty.disabled = True
    else:
        tx.disabled = False
        ty.disabled = False
c.observe(fn, names='value')
TAB['Tilted'] = widgets.VBox([c, tx, ty])

In [ ]:
m = widgets.IntText(description='Count', tooltip='Max iteration', disabled=True)
e = widgets.FloatText(description='EPS', tooltip='Desired accuracy epsilon', disabled=True)
cm = widgets.Checkbox(description='Enable')
ce = widgets.Checkbox(description='Enable')
def fn(change):
    if change.new:
        m.disabled = False
    else:
        m.disabled = True
cm.observe(fn, names='value')

def fn(change):
    if change.new:
        e.disabled = False
    else:
        e.disabled = True
ce.observe(fn, names='value')

TAB['Term criteria'] = widgets.VBox([widgets.HBox([m, cm]), widgets.HBox([e, ce])])

In [ ]:
output = widgets.Output()

tab = widgets.Tab()
#tab.titles = ['General', 'Intrinsic matrix', 'Radial', 'Tangential', 'Thin prism', 'Tilted', 'TermCriteria']

tab.children = list(TAB.values())
tab.titles = list(TAB.keys())

tab

In [ ]:
use_intrinsic_guess = widgets.Checkbox(
    value=False,
    description='Check me',
    tooltip='tooltip',
    disabled=False,
    indent=False
)

display(use_intrinsic_guess)

In [ ]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(
    object_points,
    observ_points,
    imageSize=(image_width, image_height),
    cameraMatrix=None,
    distCoeffs=None
)